Plan:
1. read in data and split by ","
2. filter and extract year and block columns from year 2012 - 2015
3. create key, value pair: block, 1 (1 is to count the crime record once)
4. reduceByKey using key block, to reduce to block, count(crime)
5. sortByKey using key count(crime), then take top 10 records

ref:
https://www.nodalpoint.com/spark-data-frames-from-csv-files-handling-headers-column-types/

In [1]:
#from pyspark import SparkContext
from pyspark import SparkContext
import csv
from operator import add
#from pyspark.sql import SQLContext




In [1]:
sc = SparkContext.getOrCreate()

In [12]:
#use when done with current pyspark context
sc.stop()

In [2]:
crimeFile = sc.textFile("Crimes_-_2001_to_present.csv")


In [4]:
crimeFile.count()

5801845

In [3]:
crimeFile.take(5)

['ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location',
 '10078659,HY267429,05/19/2015 11:57:00 PM,010XX E 79TH ST,143A,WEAPONS VIOLATION,UNLAWFUL POSS OF HANDGUN,STREET,true,false,0624,006,8,44,15,1184626,1852799,2015,05/26/2015 12:42:06 PM,41.751242944,-87.599004724,"(41.751242944, -87.599004724)"',
 '10078598,HY267408,05/19/2015 11:50:00 PM,067XX N SHERIDAN RD,3731,INTERFERENCE WITH PUBLIC OFFICER,OBSTRUCTING IDENTIFICATION,STREET,true,false,2432,024,49,1,24,1167071,1944859,2015,05/26/2015 12:42:06 PM,42.004255918,-87.660691083,"(42.004255918, -87.660691083)"',
 '10078625,HY267417,05/19/2015 11:47:00 PM,026XX E 77TH ST,2170,NARCOTICS,POSSESSION OF DRUG EQUIPMENT,STREET,true,false,0421,004,7,43,18,1195299,1854463,2015,05/26/2015 12:42:06 PM,41.755552462,-87.559839339,"(41.755552462, -87.559839339)"',
 '10078662,HY267423,05/19/2015 1

In [7]:
schemaString=crimeFile.first()
schemaString

'ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location'

In [8]:
#split by "," to create columns
fields = crimeFile.map(lambda line: line.split(","))
fields.take(10)

[['ID',
  'Case Number',
  'Date',
  'Block',
  'IUCR',
  'Primary Type',
  'Description',
  'Location Description',
  'Arrest',
  'Domestic',
  'Beat',
  'District',
  'Ward',
  'Community Area',
  'FBI Code',
  'X Coordinate',
  'Y Coordinate',
  'Year',
  'Updated On',
  'Latitude',
  'Longitude',
  'Location'],
 ['10078659',
  'HY267429',
  '05/19/2015 11:57:00 PM',
  '010XX E 79TH ST',
  '143A',
  'WEAPONS VIOLATION',
  'UNLAWFUL POSS OF HANDGUN',
  'STREET',
  'true',
  'false',
  '0624',
  '006',
  '8',
  '44',
  '15',
  '1184626',
  '1852799',
  '2015',
  '05/26/2015 12:42:06 PM',
  '41.751242944',
  '-87.599004724',
  '"(41.751242944',
  ' -87.599004724)"'],
 ['10078598',
  'HY267408',
  '05/19/2015 11:50:00 PM',
  '067XX N SHERIDAN RD',
  '3731',
  'INTERFERENCE WITH PUBLIC OFFICER',
  'OBSTRUCTING IDENTIFICATION',
  'STREET',
  'true',
  'false',
  '2432',
  '024',
  '49',
  '1',
  '24',
  '1167071',
  '1944859',
  '2015',
  '05/26/2015 12:42:06 PM',
  '42.004255918',
  '-87

In [10]:
#remove rows where block name is empty
fields = fields.filter(lambda x: len(x[3])>0) 
fields.count()

5801843

In [9]:
#select columns 2: date,3: Block, 18: year
fields2 = fields.map(lambda f: (f[3], f[17]))
fields2.take(10)

[('Block', 'Year'),
 ('010XX E 79TH ST', '2015'),
 ('067XX N SHERIDAN RD', '2015'),
 ('026XX E 77TH ST', '2015'),
 ('015XX E 62ND ST', '2015'),
 ('054XX S PRINCETON AVE', '2015'),
 ('013XX S LAWNDALE AVE', '2015'),
 ('064XX S LAFLIN ST', '2015'),
 ('021XX W NORTH AVE', '2015'),
 ('008XX N KILDARE AVE', '2015')]

In [36]:
#filter and subset to get data in recent 3 years 2013-2015
filtered = fields2.filter(lambda x: x[1] == "2013" or  x[1] == "2014" or  x[1] == "2015")
#filtered.take(10)
#count each crime record row once
pairs = filtered.map(lambda x: (x[0],1))  
#pairs.take(10)
#aggregate number of 1s or counts, group by key
#counts[0] is block, counts[1] is num of crime
counts = pairs.reduceByKey(lambda x,y: int(x) + int(y)) #count[0] is block, count[1] is num of crime
counts.take(10)

[('002XX N FRANCISCO AVE', 9),
 ('004XX N TRUMBULL AVE', 17),
 ('033XX N PITTSBURGH AVE', 7),
 ('032XX W 31ST ST', 4),
 ('057XX W CHICAGO AVE', 65),
 ('017XX N CLYBOURN AVE', 21),
 ('001XX E 83RD ST', 8),
 ('060XX S KEATING AVE', 12),
 ('064XX N LE MAI AVE', 1),
 ('020XX W SCHOOL ST', 6)]

In [43]:
#we now sort by number of crimes, so move number of crimes to position 0 to be the sort key
top10 = counts.map(lambda x: (x[1], x[0])).sortByKey(ascending = False).take(10) #this is a py list
#convert to RDD
top10rdd =  sc.parallelize(top10)
top10rdd.take(10)

[(2004, '001XX N STATE ST'),
 (1720, '0000X W TERMINAL ST'),
 (1574, '008XX N MICHIGAN AVE'),
 (1390, '076XX S CICERO AVE'),
 (1143, '0000X N STATE ST'),
 (879, '040XX W LAKE ST'),
 (876, '008XX N STATE ST'),
 (874, '064XX S DR MARTIN LUTHER KING JR DR'),
 (771, '009XX W BELMONT AVE'),
 (756, '051XX W MADISON ST')]